In [292]:
%matplotlib inline
# Dependencies and Setup
import cProfile as cPr
import gc
from math import radians, sin, cos, sqrt, atan2
from requests import Session
import time
import os
import gmaps
import json

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ratelimiter import RateLimiter as RL
from scipy.stats import linregress
from citipy.citipy import nearest_city as nc

from API.api_key import google_key as gk

def csvtoDF():
    weatherdf = pd.read_csv('../WeatherPy/outputs/worldweather.csv')
    return weatherdf
csvtoDF()

,ID,City,Country,Date,Latitude,Longitude,Max Temperature (F),Humidity,Cloudiness,Wind Speed
0,0,Loa Janan,ID,1633628739,-0.5830,117.0950,74.01,98,14,2.64
1,1,Sakakah,SA,1633628739,29.9697,40.2064,85.57,20,0,16.15
2,2,Usakos,NaN,1633628740,-22.0000,15.6000,73.42,26,0,8.61
3,3,Rundu,NaN,1633628740,-17.9333,19.7667,80.31,35,98,6.17
4,4,Kjøpsvik,NO,1633628741,68.0970,16.3742,43.18,88,91,4.41
...,...,...,...,...,...,...,...,...,...,...
1377,1377,Kandrian,PG,1633629201,-6.2167,149.5500,75.52,91,67,2.44
1378,1378,Evanston,US,1633628920,42.0411,-87.6901,72.34,94,75,4.00
1379,1379,Okhotsk,RU,1633629201,59.3833,143.3000,25.75,77,8,8.88
1380,1380,São Filipe,CV,1633629202,14.8961,-24.4956,80.22,78,22,8.41


In [242]:
def normhum():
    df = csvtoDF
    mi = df()['Humidity'].min()
    ma = df()['Humidity'].max()
    normhumid = [*df()['Humidity'].apply(lambda x: (x-mi)/(ma-mi))]
    return normhumid

In [243]:
def heatmap():
    humidity = normhum
    fig = gmaps.figure()
    gmaps.configure(api_key=gk)
    layer = gmaps.heatmap_layer(csvtoDF()[['Latitude','Longitude']],weights = humidity())
    fig.add_layer(layer)
    return fig
heatmap()

Figure(layout=FigureLayout(height='420px'))

In [244]:
heatmap().close_all()

In [267]:
def narrowdf():
    df = csvtoDF
    df1 = df()[(df()['Max Temperature (F)']>=65) & 
    (df()['Max Temperature (F)']<=80) & 
    (df()['Humidity']<=50) & 
    (df()['Country'] != 'US') & 
    (df()['Cloudiness']<50) & 
    (df()['Wind Speed']<5)]
    df2 = df1.reset_index()
    return df2
narrowdf()

,index,ID,City,Country,Date,Latitude,Longitude,Max Temperature (F),Humidity,Cloudiness,Wind Speed
0,434,434,Erzin,TR,1633628882,36.9559,36.2000,76.01,29,0,4.83
1,438,438,Nushki,PK,1633628884,29.5542,66.0215,71.58,10,0,0.49
2,560,560,Zambezi,ZM,1633628922,-13.5432,23.1047,77.49,20,34,3.40
3,605,605,Caceres,ES,1633628937,39.5220,-6.3748,73.47,34,0,3.98
4,619,619,San Juan,AR,1633628941,-31.5375,-68.5364,79.74,19,0,4.25
5,1065,1065,Barbastro,ES,1633629098,42.0356,0.1269,71.78,42,10,2.77
6,1119,1119,Paphos,CY,1633629116,34.7667,32.4167,74.98,41,0,4.61
7,1311,1311,Halifax,CA,1633628993,44.6453,-63.5724,73.35,48,20,1.99


In [268]:
def vacaycoords():
    df = narrowdf
    lats, lons = [*df()['Latitude']], [*df()['Longitude']]
    return zip(lats,lons)

def vacaycities():
    df = narrowdf
    cities, countries = [*df()['City']], [*df()['Country']]
    return zip(cities,countries)

In [293]:
def url(lat,lon):
    api_key = gk
    pre = f'https://maps.googleapis.com/maps/api/place/findplacefromtext/json'
    q = f'?input=hotel&inputtype=textquery'
    where = f'&locationbias=circle%3A5000%40{lat}%2C{lon}'
    res = f'&fields=name%2Cgeometry%2Crating%2Cformatted_address'
    key = f'&key={api_key}'
    url = ''.join([pre,q,where,res,key])
    return url

def urlist():
    build = url
    coord = vacaycoords
    urls = [build(pair[0],pair[1]) for pair in coord()]
    return urls

In [290]:
def makecalls():
    urls = urlist
    hotels = []
    with Session() as sesh:
        for u in urls():
            res = sesh.get(u).json()
            pre = res['candidates'][0]
            hotel = {'name': pre['name'],'address': pre['formatted_address'], 'rating': pre['rating'],'lat': pre['geometry']['location']['lat'], 'lng': pre['geometry']['location']['lng']}
            hotels.append(hotel)
    return hotels

In [291]:
resdict = makecalls()

In [296]:
def addcc():
    df = narrowdf
    for i in range(len(resdict)):
        lat = resdict[i]['lat']
        lng = resdict[i]['lng']
        city = nc(lat,lng).city_name.capitalize()
        country = [*df()['Country'].values][i]
        new = {'city': city, 'country': country}
        resdict[i].update(new)
    return
    
addcc()

In [299]:
def addpins():
    humidity = normhum
    api = gk
    n = len(resdict)
    info = [f'<dl><dt>Name</dt><dd>{resdict[i]["name"]}</dd><dt>Location</dt><dd>{resdict[i]["city"]}, {resdict[i]["country"]}</dd><dt>Rating</dt><dd>{resdict[i]["rating"]}</dd></dl>' for i in range(n)]
    coords = [(resdict[i]['lat'],resdict[i]['lng']) for i in range(n)]
    fig = gmaps.figure()
    gmaps.configure(api_key=api)
    layer = gmaps.heatmap_layer(csvtoDF()[['Latitude','Longitude']],weights = humidity())
    pins = gmaps.marker_layer(coords,hover_text=info,info_box_content=info,display_info_box=True)
    fig.add_layer(layer)
    fig.add_layer(pins)
    return fig

addpins()


Figure(layout=FigureLayout(height='420px'))